# model building pipeline
## things to do
* read results file.

In [1]:
from Utils import SimulationTools
from Utils import EdaTools
from Utils import Utils

## setup Parameters

In [2]:
new_sim = SimulationTools.PuckSim(
    sim_name="MM013", 
    cfile_template_name='puck_macro_template.msg.cfile', 
    keyword_main_template_name='mainTemplate.k',
    template_dir=None, 
    sim_dir="/media/martin/Stuff/research/MaterailModels"
)

## update cfile parameters and make the geometry keyword file

In [3]:
new_sim.setup_simulation(
    radius=0.01, length=0.012, num_elements=201)

## Run lsrun 
working from cmd
/home/martin/Programs/LSTC/LSDYNA/ls-dyna_smp_s_r1010_x64_suse11_pgi165 i=/media/martin/Stuff/research/MaterailModels/MM012/MM012main.k ncpu=8 memory=1000m jobid=MM012_job02

In [ ]:
new_sim.run_simulation()

In [ ]:
print(new_sim.sim_output)

## Import existing K.File

In [4]:
new_sim.import_geometry_keyword()
surf_node_ids = new_sim.get_puck_surface_nodes()

## Plot the surface nodes of the original geometry

In [5]:
%matplotlib

EdaTools.plot_3d(
    Utils.nodes_to_coord_array(
        new_sim.keyword_geom.get_nodeByID(
            surf_node_ids)))

Using matplotlib backend: Qt5Agg


## Import the results D3plot

In [6]:
new_sim.import_d3plot()

### Plot the initial and final geometry from the results file

In [10]:
EdaTools.plot_3d(
    Utils.nodes_to_coord_array(
        new_sim.d3plot.get_nodeByID(
            surf_node_ids)))

EdaTools.plot_3d(
    Utils.nodes_to_coord_array(
        new_sim.d3plot.get_nodeByID(
            surf_node_ids), timestep=-1))

### plot additional info


In [11]:
#### displacement vectors
EdaTools.plot_disp_quiver(new_sim.d3plot, node_list=surf_node_ids)

In [15]:
### nodal displacement histograms
EdaTools.plot_disp_hist(new_sim.d3plot, time_step=-1, num_bins=20)

In [16]:
#### element stress 
EdaTools.plot_stress_hist(new_sim.d3plot, time_step=-1, num_bins=20)

In [17]:
##### element strain
EdaTools.plot_strain_hist(new_sim.d3plot, time_step=-1, num_bins=20)

# fit a model to the measured displacement field

## generate a pandas frame for the data coordinate 

In [29]:
# create pandas frame
import numpy as np
import pandas as pd 

num_time_steps = new_sim.d3plot.get_nTimesteps()

data = []

# for each node on the observed surface
for nid in surf_node_ids:
    # Extract the node results
    node = new_sim.d3plot.get_nodeByID(nid)
    # extract the initial coordinates
    cords = node.get_coords()[0]
    # for each timestep
    for ts, vec in enumerate(node.get_disp()):
        # estract the node displacement and create and observation vector
        data.append(np.append(cords, [ts, *vec]))

data_df = pd.DataFrame(data, columns=["x", "y", "z", 't_index', "u", "v", "w"])

In [91]:
data_df.tail()

,x,y,z,t_index,u,v,w
55141,0.012,0.009981,-0.000622,16.0,-0.0016,0.0,0.0
55142,0.012,0.009981,-0.000622,17.0,-0.0017,0.0,0.0
55143,0.012,0.009981,-0.000622,18.0,-0.0018,0.0,0.0
55144,0.012,0.009981,-0.000622,19.0,-0.0019,0.0,0.0
55145,0.012,0.009981,-0.000622,20.0,-0.0020,0.0,0.0


In [ ]:
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import uniform
from sklearn.svm import SVR

In [111]:
X = data_df[["x", "y", "z", 't_index']]
y = data_df["u"]
clf = SVR(
#     C=1.0,
#     degree=3,
#     epsilon=0.1,
    gamma='scale',
    kernel='rbf',
    tol=0.00000001
)
# clf.fit(X, y)

In [112]:
distributions = dict(C=uniform(loc=0, scale=4), degree=[3, 4, 5], epsilon=uniform(loc=0, scale=4))
clf = RandomizedSearchCV(
    clf, 
    distributions, 
    random_state=0, 
    n_iter= 30, 
    scoring='explained_variance',
    n_jobs=4,
    cv=5
)
search = clf.fit(X, y)
search.best_params_

{'C': 2.195254015709299, 'degree': 4, 'epsilon': 3.3770629943240693}

In [113]:
search.best_score_


4.441214217392187e-17

In [114]:
search.predict([[0.012, 0.009981, 0.000622, 20.0]])

array([-0.001])

In [90]:
search.best_estimator_

SVR(C=2.195254015709299, cache_size=200, coef0=0.0, degree=4,
    epsilon=3.3770629943240693, gamma='auto', kernel='rbf', max_iter=-1,
    shrinking=True, tol=1e-06, verbose=False)

In [115]:
search.best_estimator_.predict(X)

array([-0.001, -0.001, -0.001, ..., -0.001, -0.001, -0.001])